In [ ]:
# Loading in required libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Read in the Nobel Prize data
nobel = pd.read_csv('data/nobel.csv')
nobel.head()

# Store and display the most commonly awarded gender and birth country in requested variables
top_gender = nobel['sex'].value_counts().idxmax()
top_gender_value=nobel["sex"].value_counts().max()
top_country = nobel['birth_country'].value_counts().idxmax()
top_country_value= nobel["birth_country"].value_counts().max()

nobel["decade"]=np.floor(nobel["year"]/10)*10
decade_winners=nobel["decade"].value_counts()

decade_winners_yankies=nobel.query('birth_country=="United States of America"')["decade"].value_counts()
decade_winners_yankies_prop=decade_winners_yankies/decade_winners
max_decade_usa=decade_winners_yankies_prop.idxmax()
max_decade_usa_value=decade_winners_yankies_prop.max()
plt.plot(decade_winners_yankies_prop.index,decade_winners_yankies_prop.values)
plt.title("Proportion of Yankee Laureates per decade")
plt.xlabel("Decade")
plt.ylabel("% of USA Laureates")

print("The decade with more yankee laureates proportionally was "+str(round(max_decade_usa))+", the proportion was "+str(round(max_decade_usa_value,1))+"%.")

#decade_winners_category=nobel.groupby(["decade","category"]).count().sort_values(by=["decade","category"], ascending=[False,False])
decade_winners_cat=nobel[["decade","category"]].value_counts().rename("count_all")
decade_winners_cat_women=nobel.query('sex=="Female"')[["decade","category"]].value_counts().rename("count_women")

#If they had the same size, but they do not.
decade_winners_cat_women.shape==decade_winners_cat.shape
decade_winners_cat_women_prop=decade_winners_cat_women/decade_winners_cat
max_decade_winners_cat_women_prop=decade_winners_cat_women_prop.idxmax()
max_decade_winners_cat_women_prop_value=decade_winners_cat_women_prop.max()

#Different sizes, join is required; joins do not work with Series, we need pd dataframes
decade_winners_cat=decade_winners_cat.reset_index()
decade_winners_cat_women=decade_winners_cat_women.reset_index()
merged_data=pd.merge(decade_winners_cat,decade_winners_cat_women, on=["decade","category"],suffixes=["_all","_women"], how="left").fillna(0)

merged_data["women_prop"]=merged_data["count_women"]/merged_data["count_all"]
max_decade_winners_cat_women_prop_value=merged_data["women_prop"].max()
max_decade_winners_cat_women_prop=merged_data.loc[merged_data["women_prop"]==merged_data["women_prop"].max(),["decade","category"]]

print("The decade with more women laureates proportionally was "+str(round(max_decade_winners_cat_women_prop.values[0][0]))+" in the category "+max_decade_winners_cat_women_prop.values[0][1]+" the proportion was "+str(max_decade_winners_cat_women_prop_value)+"%.")

first_women_year=nobel.loc[nobel["sex"]=="Female",["year"]].sort_values(by=["year"], ascending=[True]).min().values[0]
first_woman_name=nobel.loc[(nobel["sex"]=="Female") & (nobel["year"]==first_women_year),["full_name"]].values[0][0]
first_woman_category=nobel.loc[(nobel["sex"]=="Female") & (nobel["year"]==first_women_year),["category"]].values[0][0]
print("The year in which we find the first female Laureate is "+str(round(first_women_year))+" in the category "+first_woman_category+" her name was "+first_woman_name+".")

repeat_list=nobel.loc[nobel.duplicated(subset=["full_name"]),"full_name"]
#or using direct indexing: repeat_list=nobel[nobel.duplicated(subset=["full_name"])]["full_name"]
duplicates=nobel.duplicated(subset=["full_name"]).sum()
print("There are "+str(duplicates)+" institutions or individuals that won the prize more than once; those are: ")
print(repeat_list)